Check CUDA

In [1]:
import torch

print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name(0))
else:
    print("CUDA not available — still CPU only")

print(torch.__version__)  # should be 2.6.0 or newer

     

CUDA Available: True
Device: NVIDIA GeForce GTX 1080
2.6.0+cu126


In [2]:
import pandas as pd
df = pd.read_csv('dataset.csv')

In [3]:
# Check distribution
print(df['label'].value_counts())

# Shuffle dataset 
df = df.sample(frac=1, random_state=42)

label
4998    1
0       1
1       1
2       1
3       1
       ..
11      1
10      1
9       1
8       1
7       1
Name: count, Length: 4999, dtype: int64


Tokenization & Input Formatting

In [4]:
# Concatenate relevant fields for input
texts = (
    df['medical_specialty'].astype(str) + " | " +
    df['sample_name'].astype(str) + " | " +
    df['transcription'].astype(str) + " | " +
    df['keywords'].astype(str)
).tolist()



Split Dataset

In [5]:
from sklearn.model_selection import train_test_split

# Split into training and testing (e.g. 80% train, 20% test)
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, df['label'].tolist(), test_size=0.2, random_state=42
)


Tokenize 

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('dmis-lab/biobert-v1.1')

# Tokenize
def tokenize(texts):
    return tokenizer(
        texts,
        padding='max_length',
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )

train_encodings = tokenize(train_texts)
test_encodings = tokenize(test_texts)


d:\AI\Diagnoses Prediction\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Train Test Split

In [7]:
from torch.utils.data import Dataset

# Define a custom Dataset class
class MedicalDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }
    
# Create train and test datasets
train_dataset = MedicalDataset(train_encodings, train_labels)
test_dataset = MedicalDataset(test_encodings, test_labels)

Fine-Tune BioBert

In [10]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)

# Load pre-trained model
num_labels = df['label'].nunique()
model = AutoModelForSequenceClassification.from_pretrained('dmis-lab/biobert-v1.1', num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    eval_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",

)

# Define evaluation metrics
def compute_metrics(p):
    preds = p.predictions.argmax(axis=1)
    return {
        'accuracy': accuracy_score(p.label_ids, preds),
        'f1': f1_score(p.label_ids, preds)
    }

# Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
